In [2]:
import numpy as np
from scipy.linalg import eigh
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import robust_laplacian
from Mesh import Mesh
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.sparse import csr_matrix, dia_matrix, csc_matrix
from scipy import sparse
from tqdm import trange

In [ ]:
# ============ DEVICE SETUP ============
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# ============ NETWORK ARCHITECTURE ============
class Sin(nn.Module):
    """Sine activation function"""
    def forward(self, x):
        return torch.sin(x)


class EigenfunctionNN(nn.Module):
    """
    Neural network to learn eigenfunctions on point clouds.
    Input: 3D coordinates (x, y, z)
    Output: eigenfunction value u(x,y,z) and eigenvalue λ
    """
    def __init__(self, hidden_dim=64, input_dim=3, initial_eigenvalue=0.0):
        super().__init__()
        self.activation = Sin()
        
        # Learnable eigenvalue with better initialization
        self.eigenvalue_layer = nn.Linear(1, 1, bias=False)
        with torch.no_grad():
            self.eigenvalue_layer.weight.fill_(initial_eigenvalue)
        
        # Network layers - concatenate eigenvalue at each layer
        self.fc1 = nn.Linear(input_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc2 = nn.Linear(hidden_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc3 = nn.Linear(hidden_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc4 = nn.Linear(hidden_dim + 1, 1)  # +1 for eigenvalue
        
    def forward(self, x):
        """
        Args:
            x: (N, 3) point cloud coordinates
        Returns:
            u: (N, 1) eigenfunction values
            eigenvalue: scalar learnable eigenvalue
        """
        # Learn eigenvalue and broadcast to match batch size
        eigenvalue = torch.abs(self.eigenvalue_layer(torch.ones(1, 1).to(x.device)))
        eigenvalue_expanded = eigenvalue.expand(x.shape[0], 1)  # (N, 1)
        
        # Forward pass - concatenate eigenvalue at each layer
        h = torch.cat([x, eigenvalue_expanded], dim=1)  # (N, input_dim+1)
        h = self.activation(self.fc1(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        h = self.activation(self.fc2(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        h = self.activation(self.fc3(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        u = self.fc4(h)
        
        return u, eigenvalue


# ============ LOSS COMPUTATION ============
def compute_eigenvalue_loss(u, eigenvalue, L, M, X, device):
    """
    Compute residual for Lu = λMu using discrete operators.
    
    Args:
        u: (N, 1) predicted eigenfunction values
        eigenvalue: scalar predicted eigenvalue
        L: (N, N) Laplacian matrix (scipy sparse: csr, csc, dia, etc.)
        M: (N, N) Mass matrix (scipy sparse: csr, csc, dia, etc.)
        X: (N, 3) point cloud coordinates
        device: torch device
    
    Returns:
        loss: MSE of residual ||Lu - λMu||²
    """
    u_flat = u.squeeze()  # (N,)
    
    # Convert sparse matrices to torch sparse tensors if needed
    if sparse.issparse(L):
        L_torch = sparse_to_torch(L, device)
        M_torch = sparse_to_torch(M, device)
    else:
        L_torch = L
        M_torch = M
    
    # Compute Lu and λMu
    Lu = torch.sparse.mm(L_torch, u_flat.unsqueeze(1)).squeeze()
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    lMu = eigenvalue * Mu
    
    # Residual loss
    residual = Lu - lMu
    loss = torch.mean(residual ** 2)
    
    return loss, Lu, Mu


def compute_normalization_loss(u, M, device):
    """
    Enforce u^T M u = 1 (mass-matrix normalization).
    """
    u_flat = u.squeeze()
    
    if sparse.issparse(M):
        M_torch = sparse_to_torch(M, device)
    else:
        M_torch = M
    
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    norm_squared = torch.dot(u_flat, Mu)
    
    # Penalize deviation from unit norm
    loss = (norm_squared - 1.0) ** 2
    
    return loss


def compute_orthogonality_loss(u, previous_eigenfunctions, M, device):
    """
    Enforce u ⊥ u_i for all previously found eigenfunctions.
    Uses M-orthogonality: u^T M u_i = 0
    """
    if len(previous_eigenfunctions) == 0:
        return torch.tensor(0.0, device=device)
    
    u_flat = u.squeeze()
    ortho_loss = torch.tensor(0.0, device=device)
    
    if sparse.issparse(M):
        M_torch = sparse_to_torch(M, device)
    else:
        M_torch = M
    
    for u_prev in previous_eigenfunctions:
        u_prev_flat = u_prev.squeeze()
        # Compute u^T M u_prev
        Mu_prev = torch.sparse.mm(M_torch, u_prev_flat.unsqueeze(1)).squeeze()
        overlap = torch.dot(u_flat, Mu_prev)
        ortho_loss += overlap ** 2
    
    return ortho_loss


# ============ UTILITY FUNCTIONS ============
def sparse_to_torch(sparse_matrix, device):
    """Convert scipy sparse matrix to torch sparse tensor."""
    # Handle any scipy sparse format
    if sparse.issparse(sparse_matrix):
        coo = sparse_matrix.tocoo()
    else:
        raise ValueError(f"Expected scipy sparse matrix, got {type(sparse_matrix)}")
    
    indices = torch.LongTensor(np.vstack((coo.row, coo.col)))
    values = torch.FloatTensor(coo.data)
    shape = coo.shape
    return torch.sparse_coo_tensor(indices, values, shape).to(device)


def initialize_weights(m):
    """Reinitialize network weights for finding next eigenfunction."""
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


# ============ MAIN TRAINING FUNCTION ============
def train_eigenvalue_pinn(X, L, M, hidden_dim=64, epochs=20000, 
                          lr=1e-3, num_eigenfunctions=5, 
                          convergence_threshold=1e-7,
                          ortho_weight=1.0):
    """print(f"The true Eigenvalues are: {np.array2string(eigvals[:10], formatter={'float': lambda x: f'{x:.3f}'})}")
print(f"The predicted Eigenvalues are: {np.array2string(np.array(eigenvalues), formatter={'float': lambda x: f'{x:.3f}'})}")
    Train PINN to solve Lu = λMu eigenvalue problem.
    
    Args:
        X: (N, 3) point cloud coordinates
        L: (N, N) Laplacian matrix (scipy sparse: csr, csc, dia, etc.)
        M: (N, N) Mass matrix (scipy sparse: csr, csc, dia, etc.)
        hidden_dim: Hidden layer dimension
        epochs: Training epochs per eigenfunction
        lr: Learning rate
        num_eigenfunctions: Number of eigenfunctions to find
        convergence_threshold: Threshold for detecting convergence
        ortho_weight: Weight for orthogonality loss (increase if eigenfunctions overlap)
    
    Returns:
        eigenvalues: List of found eigenvalues
        eigenfunctions: List of (N, 1) eigenfunctions
        loss_history: Training loss history
    """
    
    # Convert inputs to torch (handle both numpy arrays and torch tensors)
    if isinstance(X, np.ndarray):
        X_torch = torch.FloatTensor(X).to(device)
    elif isinstance(X, torch.Tensor):
        X_torch = X.float().to(device)  # Ensure float32
    else:
        raise ValueError(f"X must be numpy array or torch tensor, got {type(X)}")
    
    X_torch.requires_grad = True
    
    # Storage for results
    eigenvalues = []
    eigenfunctions = []
    all_models = []
    loss_history = {'total': [], 'eigenvalue': [], 'normalization': [], 'orthogonality': []}
    
    print(f"Training on device: {device}")
    print(f"Point cloud size: {X.shape[0]} points")
    print(f"Matrix format: L is {type(L).__name__}, M is {type(M).__name__}")
    
    # ============ ITERATIVE EIGENFUNCTION DISCOVERY ============
    for eig_idx in range(num_eigenfunctions):
        print(f"\n{'='*60}")
        print(f"Finding eigenfunction {eig_idx + 1}/{num_eigenfunctions}")
        print(f"{'='*60}")
        
        # Initialize network with progressively larger eigenvalue guess
        # For Laplacian: smallest eigenvalue is 0
        if eig_idx == 0:
            initial_eigenvalue = 0.0  # First eigenvalue for Laplacian
        elif eig_idx > 0:
            # Use previous eigenvalue as lower bound + small increment
            initial_eigenvalue = eigenvalues[-1] + 0.15
        else:
            initial_eigenvalue = eig_idx * 0.2
        
        model = EigenfunctionNN(hidden_dim=hidden_dim, input_dim=X.shape[1], 
                               initial_eigenvalue=initial_eigenvalue).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.999, 0.9999))
        
        best_model = None
        best_loss = float('inf')
        loss_slope_history = []
        
        for epoch in range(epochs):
            optimizer.zero_grad()
            
            # Forward pass
            u, eigenvalue = model(X_torch)
            
            # Compute losses
            eig_loss, Lu, Mu = compute_eigenvalue_loss(u, eigenvalue, L, M, X_torch, device)
            norm_loss = compute_normalization_loss(u, M, device)
            ortho_loss = compute_orthogonality_loss(u, eigenfunctions, M, device)
            
            # Total loss with weighting
            total_loss = eig_loss + norm_loss + ortho_weight * ortho_loss
            
            # Backward pass
            total_loss.backward()
            optimizer.step()
            
            # Track loss slope for convergence detection
            loss_slope_history.append(eig_loss.item())
            if len(loss_slope_history) > 1000:
                loss_slope_history.pop(0)
                slope = np.mean(np.diff(loss_slope_history))
            else:
                slope = 0.0
            
            # Save best model
            if eig_loss.item() < best_loss:
                best_loss = eig_loss.item()
                best_model = copy.deepcopy(model)
            
            # Logging
            if epoch % 500 == 0:
                print(f"Epoch {epoch:5d} | λ={eigenvalue.item():.6f} | "
                      f"Eig loss={eig_loss.item():.2e} | Norm loss={norm_loss.item():.2e} | "
                      f"Ortho loss={ortho_loss.item():.2e} | Slope={slope:.2e}")
            
            # Store history
            loss_history['total'].append(total_loss.item())
            loss_history['eigenvalue'].append(eig_loss.item())
            loss_history['normalization'].append(norm_loss.item())
            loss_history['orthogonality'].append(ortho_loss.item())
            
            # Check for convergence and reinitialize if stuck
            if epoch > 5000 and len(loss_slope_history) == 1000:
                if abs(slope) < convergence_threshold:
                    print(f"Converged at epoch {epoch}!")
                    break
        
        # Store results for this eigenfunction
        with torch.no_grad():
            u_final, eigenvalue_final = best_model(X_torch)
            eigenvalues.append(eigenvalue_final.item())
            eigenfunctions.append(u_final.detach())
            all_models.append(best_model)
        
        print(f"\nFound eigenvalue: λ_{eig_idx} = {eigenvalue_final.item():.6f}")
    
    return eigenvalues, eigenfunctions, all_models, loss_history


In [18]:
"""

THIS ONE ACTUALLY WORKS OK

"""



# ============ DEVICE SETUP ============
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# ============ NETWORK ARCHITECTURE ============
class Sin(nn.Module):
    """Sine activation function"""
    def forward(self, x):
        return torch.sin(x)


class EigenfunctionNN(nn.Module):
    """
    Neural network to learn eigenfunctions on point clouds.
    Input: 3D coordinates (x, y, z)
    Output: eigenfunction value u(x,y,z) and eigenvalue λ
    """
    def __init__(self, hidden_dim=64, input_dim=3, initial_eigenvalue=0.0):
        super().__init__()
        self.activation = Sin()
        
        # Learnable eigenvalue with better initialization
        self.eigenvalue_layer = nn.Linear(1, 1, bias=False)
        with torch.no_grad():
            self.eigenvalue_layer.weight.fill_(initial_eigenvalue)
        
        # Network layers - concatenate eigenvalue at each layer
        self.fc1 = nn.Linear(input_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc2 = nn.Linear(hidden_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc3 = nn.Linear(hidden_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc4 = nn.Linear(hidden_dim + 1, 1)  # +1 for eigenvalue
        
    def forward(self, x):
        """
        Args:
            x: (N, 3) point cloud coordinates
        Returns:
            u: (N, 1) eigenfunction values
            eigenvalue: scalar learnable eigenvalue
        """
        # Learn eigenvalue and broadcast to match batch size
        eigenvalue = torch.abs(self.eigenvalue_layer(torch.ones(1, 1).to(x.device)))
        eigenvalue_expanded = eigenvalue.expand(x.shape[0], 1)  # (N, 1)
        
        # Forward pass - concatenate eigenvalue at each layer
        h = torch.cat([x, eigenvalue_expanded], dim=1)  # (N, input_dim+1)
        h = self.activation(self.fc1(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        h = self.activation(self.fc2(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        h = self.activation(self.fc3(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        u = self.fc4(h)
        
        return u, eigenvalue


# ============ LOSS COMPUTATION ============
def compute_eigenvalue_loss(u, eigenvalue, L_torch, M_torch):
    """
    Compute residual for Lu = λMu using discrete operators.
    Returns:
        loss: MSE of residual ||Lu - λMu||²
    """
    u_flat = u.squeeze()  # (N,)
    
    # Compute Lu and λMu
    Lu = torch.sparse.mm(L_torch, u_flat.unsqueeze(1)).squeeze()
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    residual = Lu - eigenvalue * Mu
    
    return torch.mean(residual ** 2), Lu, Mu


def compute_normalization_loss(u, M_torch):
    """
    Enforce u^T M u = 1 (mass-matrix normalization).
    """
    u_flat = u.squeeze()    
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    norm_squared = torch.dot(u_flat, Mu)

    return (norm_squared - 1.0) ** 2


def compute_orthogonality_loss(u, previous_eigenfunctions, M_torch):
    """
    Enforce u ⊥ u_i for all previously found eigenfunctions.
    Uses M-orthogonality: u^T M u_i = 0
    """
    if not previous_eigenfunctions:
        return torch.tensor(0.0, device=M_torch.device)
    
    u_flat = u.squeeze()
    ortho_loss = torch.tensor(0.0, device=M_torch.device)
    
    
    for u_prev in previous_eigenfunctions:
        u_prev_flat = u_prev.squeeze()
        # Compute u^T M u_prev
        Mu_prev = torch.sparse.mm(M_torch, u_prev_flat.unsqueeze(1)).squeeze()
        overlap = torch.dot(u_flat, Mu_prev)
        ortho_loss += overlap ** 2
    
    return ortho_loss


# ============ UTILITY FUNCTIONS ============
def sparse_to_torch(sparse_matrix, device):
    """Convert scipy sparse matrix to torch sparse tensor."""
    # Handle any scipy sparse format
    if sparse.issparse(sparse_matrix):
        coo = sparse_matrix.tocoo()
    else:
        raise ValueError(f"Expected scipy sparse matrix, got {type(sparse_matrix)}")
    
    indices = torch.LongTensor(np.vstack((coo.row, coo.col)))
    values = torch.FloatTensor(coo.data)
    shape = coo.shape
    return torch.sparse_coo_tensor(indices, values, shape).to(device)


def initialize_weights(m):
    """Reinitialize network weights for finding next eigenfunction."""
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


# ============ MAIN TRAINING FUNCTION ============
def train_eigenvalue_pinn(X, L, M, hidden_dim=64, epochs=20000, 
                          lr=1e-3, num_eigenfunctions=5, 
                          convergence_threshold=1e-7,
                          ortho_weight=1.0):
    """print(f"The true Eigenvalues are: {np.array2string(eigvals[:10], formatter={'float': lambda x: f'{x:.3f}'})}")
print(f"The predicted Eigenvalues are: {np.array2string(np.array(eigenvalues), formatter={'float': lambda x: f'{x:.3f}'})}")
    Train PINN to solve Lu = λMu eigenvalue problem.
    
    Args:
        X: (N, 3) point cloud coordinates
        L: (N, N) Laplacian matrix (scipy sparse: csr, csc, dia, etc.)
        M: (N, N) Mass matrix (scipy sparse: csr, csc, dia, etc.)
        hidden_dim: Hidden layer dimension
        epochs: Training epochs per eigenfunction
        lr: Learning rate
        num_eigenfunctions: Number of eigenfunctions to find
        convergence_threshold: Threshold for detecting convergence
        ortho_weight: Weight for orthogonality loss (increase if eigenfunctions overlap)
    
    Returns:
        eigenvalues: List of found eigenvalues
        eigenfunctions: List of (N, 1) eigenfunctions
        loss_history: Training loss history
    """
    
    # Prepare inputs
    X_torch = torch.as_tensor(X, dtype=torch.float32, device=device)
    X_torch.requires_grad = True

    # Pre-convert sparse matrices
    L_torch = sparse_to_torch(L, device) if sparse.issparse(L) else L.to(device)
    M_torch = sparse_to_torch(M, device) if sparse.issparse(M) else M.to(device)
    
    # Storage for results
    eigenvalues = []
    eigenfunctions = []
    all_models = []
    loss_history = {'total': [], 'eig': [], 'norm': [], 'ortho': []}
    
    print(f"Training on device: {device}")
    print(f"Point cloud size: {X.shape[0]} points")
    print(f"Matrix format: L is {type(L).__name__}, M is {type(M).__name__}")
    
    # ============ ITERATIVE EIGENFUNCTION DISCOVERY ============
    for eig_idx in range(num_eigenfunctions):
        print(f"\n{'='*60}")
        print(f"Finding eigenfunction {eig_idx + 1}/{num_eigenfunctions}")
        print(f"{'='*60}")
        
        # Initialize network with progressively larger eigenvalue guess
        # For Laplacian: smallest eigenvalue is 0
        if eig_idx == 0:
            initial_eigenvalue = 0.0  # First eigenvalue for Laplacian
        elif eig_idx > 0:
            # Use previous eigenvalue as lower bound + small increment
            initial_eigenvalue = eigenvalues[-1] + 0.15
        else:
            initial_eigenvalue = eig_idx * 0.2
        
        model = EigenfunctionNN(hidden_dim=hidden_dim, input_dim=X.shape[1], 
                               initial_eigenvalue=initial_eigenvalue).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.999, 0.9999))
        
        best_model = None
        best_loss = float('inf')
        ema_slope = 1.0
        prev_loss = None
        
        for epoch in trange(epochs, desc=f"Eigen {eig_idx+1}/{num_eigenfunctions}"):
            optimizer.zero_grad()
            
            # Forward pass
            u, eigenvalue = model(X_torch)
            
            # Compute losses
            eig_loss, _, _ = compute_eigenvalue_loss(u, eigenvalue, L_torch, M_torch)
            norm_loss = compute_normalization_loss(u, M_torch)
            ortho_loss = compute_orthogonality_loss(u, eigenfunctions, M_torch)
            
            # Total loss with weighting
            total_loss = eig_loss + norm_loss + ortho_weight * ortho_loss
            
            # Backward pass
            total_loss.backward()
            optimizer.step()
            
            # Convergence tracking
            if prev_loss is not None:
                ema_slope = 0.75 * ema_slope + 0.25 * abs(prev_loss - eig_loss.item())
            prev_loss = eig_loss.item()

            if ema_slope < convergence_threshold and epoch > 2000:
                print(f"Converged at epoch {epoch}")
                break

            # Save best
            if eig_loss.item() < best_loss:
                best_loss = eig_loss.item()
                best_model = copy.deepcopy(model)

            # Logging every 500 epochs
            if epoch % 500 == 0:
                print(f"Epoch {epoch:5d} | λ={eigenvalue.item():.6f} | "
                      f"Eig={eig_loss.item():.2e} | Norm={norm_loss.item():.2e} | "
                      f"Ortho={ortho_loss.item():.2e} | EMA slope={ema_slope:.2e}")

            # Lightweight history
            if epoch % 100 == 0:
                loss_history['total'].append(total_loss.item())
                loss_history['eig'].append(eig_loss.item())
                loss_history['norm'].append(norm_loss.item())
                loss_history['ortho'].append(ortho_loss.item())

        # Store results
        with torch.no_grad():
            u_final, eigenvalue_final = best_model(X_torch)
            eigenvalues.append(eigenvalue_final.item())
            eigenfunctions.append(u_final.detach())
            all_models.append(best_model)

        print(f"\nFound eigenvalue: λ_{eig_idx} = {eigenvalue_final.item():.6f}")
    
    return eigenvalues, eigenfunctions, all_models, loss_history


In [20]:
m = Mesh('bunny.obj')

centroid = m.verts.mean(0)
std_max = m.verts.std(0).max()

X = (m.verts - centroid)/std_max

m = Mesh(verts = X, connectivity = m.connectivity)

L, M = robust_laplacian.point_cloud_laplacian(X)

print('Computing Laplacian')
K_igl, M_igl = m.computeLaplacian()

# following Finite Elements methodology 
# K is stiffness matrix, M is mass matrix
# The problem to solve becomes 
# K*u = lambda * M*u
print('Computing eigen values')
eigvals, eigvecs = eigh(K_igl,M_igl)

# send all relevant numpy arrays to torch tensors
K_ = torch.from_numpy(K_igl).float().to(device)
M_ = torch.from_numpy(M_igl).float().to(device)
X_ = torch.from_numpy(m.verts).float().to(device)



eigenvalues, eigenfunctions, models, history = train_eigenvalue_pinn(
    X_, K_, M_, 
    hidden_dim=128, 
    epochs=5000, 
    lr=1e-3, 
    num_eigenfunctions=5
)

print("\n" + "="*60)
print("RESULTS")
print("="*60)
for i, lam in enumerate(eigenvalues):
    print(f"λ_{i} = {lam:.6f}")

Computing Laplacian
Computing eigen values
Training on device: cuda
Point cloud size: 2503 points
Matrix format: L is Tensor, M is Tensor

Finding eigenfunction 1/5


Eigen 1/5:   1%|          | 39/5000 [00:00<00:12, 384.44it/s]

Epoch     0 | λ=0.000000 | Eig=1.25e-05 | Norm=6.37e-02 | Ortho=0.00e+00 | EMA slope=1.00e+00


Eigen 1/5:  11%|█▏        | 572/5000 [00:01<00:09, 486.38it/s]

Epoch   500 | λ=0.000000 | Eig=2.59e-05 | Norm=2.79e-02 | Ortho=0.00e+00 | EMA slope=2.85e-07


...

Epoch  1000 | λ=0.392971 | Eig=1.53e-05 | Norm=8.17e-03 | Ortho=2.26e-02 | EMA slope=1.52e-07


Eigen 5/5:  31%|███       | 1541/5000 [00:04<00:09, 376.30it/s]

Epoch  1500 | λ=0.339904 | Eig=1.57e-05 | Norm=9.32e-03 | Ortho=2.43e-02 | EMA slope=5.59e-08


Eigen 5/5:  40%|████      | 2001/5000 [00:05<00:08, 361.29it/s]

Epoch  2000 | λ=0.303474 | Eig=1.47e-05 | Norm=8.37e-03 | Ortho=1.67e-02 | EMA slope=1.95e-08
Converged at epoch 2001

Found eigenvalue: λ_4 = 0.599559

RESULTS
λ_0 = 0.000000
λ_1 = 0.151000
λ_2 = 0.302000
λ_3 = 0.453000
λ_4 = 0.599559


In [5]:
eigenvalues

[0.0,
 0.14900000393390656,
 0.23257632553577423,
 0.3801209032535553,
 0.5295212864875793,
 0.6682814955711365,
 0.8164870142936707,
 0.9114035964012146,
 1.0432565212249756,
 1.1872462034225464,
 1.3285917043685913,
 1.4192572832107544,
 1.479326844215393,
 1.6303268671035767,
 1.7489262819290161,
 1.886279582977295,
 2.008985757827759,
 2.1078410148620605,
 2.2441799640655518,
 2.305201768875122,
 2.432590961456299,
 2.5144846439361572,
 2.6198272705078125,
 2.655687093734741,
 2.7635750770568848,
 2.839317560195923,
 2.986765146255493,
 3.1014328002929688,
 3.231534481048584,
 3.28542160987854,
 3.336639404296875,
 3.4122958183288574,
 3.5568699836730957,
 3.69858455657959,
 3.761384963989258,
 3.8405449390411377,
 3.943140745162964,
 4.037494659423828,
 4.180729866027832,
 4.321310520172119,
 4.421263694763184,
 4.499535083770752,
 4.598731517791748,
 4.745480060577393,
 4.890344619750977,
 5.039665222167969,
 5.177450656890869,
 5.362771034240723,
 5.5051445960998535,
 5.67622566

In [16]:
"""

THIS ONE ACTUALLY WORKS OK

"""



# ============ DEVICE SETUP ============
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# ============ NETWORK ARCHITECTURE ============
class Sin(nn.Module):
    """Sine activation function"""
    def forward(self, x):
        return torch.sin(x)


class EigenfunctionNN(nn.Module):
    """
    Neural network to learn eigenfunctions on point clouds.
    Input: 3D coordinates (x, y, z)
    Output: eigenfunction value u(x,y,z) and eigenvalue λ
    """
    def __init__(self, hidden_dim=64, input_dim=3, initial_eigenvalue=0.0):
        super().__init__()
        self.activation = Sin()
        
        # Learnable eigenvalue with better initialization
        self.eigenvalue_layer = nn.Linear(1, 1, bias=False)
        with torch.no_grad():
            self.eigenvalue_layer.weight.fill_(initial_eigenvalue)
        
        # Network layers - concatenate eigenvalue at each layer
        self.fc1 = nn.Linear(input_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc2 = nn.Linear(hidden_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc3 = nn.Linear(hidden_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc4 = nn.Linear(hidden_dim + 1, 1)  # +1 for eigenvalue
        
    def forward(self, x):
        """
        Args:
            x: (N, 3) point cloud coordinates
        Returns:
            u: (N, 1) eigenfunction values
            eigenvalue: scalar learnable eigenvalue
        """
        # Learn eigenvalue and broadcast to match batch size
        eigenvalue = torch.abs(self.eigenvalue_layer(torch.ones(1, 1).to(x.device)))
        eigenvalue_expanded = eigenvalue.expand(x.shape[0], 1)  # (N, 1)
        
        # Forward pass - concatenate eigenvalue at each layer
        h = torch.cat([x, eigenvalue_expanded], dim=1)  # (N, input_dim+1)
        h = self.activation(self.fc1(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        h = self.activation(self.fc2(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        h = self.activation(self.fc3(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        u = self.fc4(h)
        
        return u, eigenvalue


# ============ LOSS COMPUTATION ============
def compute_eigenvalue_loss(u, eigenvalue, L_torch, M_torch):
    """
    Compute residual for Lu = λMu using discrete operators.
    Returns:
        loss: MSE of residual ||Lu - λMu||²
    """
    u_flat = u.squeeze()  # (N,)
    
    # Compute Lu and λMu
    Lu = torch.sparse.mm(L_torch, u_flat.unsqueeze(1)).squeeze()
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    residual = Lu - eigenvalue * Mu
    
    return torch.mean(residual ** 2), Lu, Mu


def compute_normalization_loss(u, M_torch):
    """
    Enforce u^T M u = 1 (mass-matrix normalization).
    """
    u_flat = u.squeeze()    
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    norm_squared = torch.dot(u_flat, Mu)

    return (norm_squared - 1.0) ** 2


def compute_orthogonality_loss(u, previous_eigenfunctions, M_torch):
    """
    Enforce u ⊥ u_i for all previously found eigenfunctions.
    Uses M-orthogonality: u^T M u_i = 0
    """
    if not previous_eigenfunctions:
        return torch.tensor(0.0, device=M_torch.device)
    
    u_flat = u.squeeze()
    ortho_loss = torch.tensor(0.0, device=M_torch.device)
    
    
    for u_prev in previous_eigenfunctions:
        u_prev_flat = u_prev.squeeze()
        # Compute u^T M u_prev
        Mu_prev = torch.sparse.mm(M_torch, u_prev_flat.unsqueeze(1)).squeeze()
        overlap = torch.dot(u_flat, Mu_prev)
        ortho_loss += overlap ** 2
    
    return ortho_loss


# ============ UTILITY FUNCTIONS ============
def sparse_to_torch(sparse_matrix, device):
    """Convert scipy sparse matrix to torch sparse tensor."""
    # Handle any scipy sparse format
    if sparse.issparse(sparse_matrix):
        coo = sparse_matrix.tocoo()
    else:
        raise ValueError(f"Expected scipy sparse matrix, got {type(sparse_matrix)}")
    
    indices = torch.LongTensor(np.vstack((coo.row, coo.col)))
    values = torch.FloatTensor(coo.data)
    shape = coo.shape
    return torch.sparse_coo_tensor(indices, values, shape).to(device)


def initialize_weights(m):
    """Reinitialize network weights for finding next eigenfunction."""
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


# ============ MAIN TRAINING FUNCTION ============
def train_eigenvalue_pinn_adaptive(
    X, L, M,
    hidden_dim=64,
    epochs=20000,
    lr=1e-3,
    num_eigenfunctions=5,
    convergence_threshold=1e-7,
    ortho_weight=25.0,
    minibatch_size=None,
    perturbation_factor=0.002
):
    """
    Adaptive PINN for solving Lu = λMu using a single network:
      - Stores converged eigenfunctions before reinitialization
      - Adaptive in-loop reinitialization (like Schrödinger PINN)
      - Point perturbation + minibatching
      - Orthogonality enforcement
    """
    # ======== Setup ========
    X_torch = torch.as_tensor(X, dtype=torch.float32, device=device)
    N = X_torch.shape[0]
    L_torch = sparse_to_torch(L, device) if sparse.issparse(L) else L.to(device)
    M_torch = sparse_to_torch(M, device) if sparse.issparse(M) else M.to(device)
    domain_scale = (X_torch.max(0).values - X_torch.min(0).values).mean()

    if minibatch_size is None or minibatch_size > N:
        minibatch_size = N
    num_batches = max(1, N // minibatch_size)

    # ======== Initialize network ========
    model = EigenfunctionNN(hidden_dim=hidden_dim, input_dim=X.shape[1]).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.999, 0.9999))

    # ======== Storage ========
    eigenfunctions = []
    eigenvalues = []
    all_models = []
    loss_history = {'total': [], 'eig': [], 'norm': [], 'ortho': []}

    eig_counter = 0  # Tracks how many eigenfunctions have been found
    ema_slope, prev_loss = 1.0, None

    print(f"Training on device: {device}, Total points: {N}, Minibatch size: {minibatch_size}")

    # ======== Training Loop ========
    for epoch in range(epochs):
        # ======== Perturb and shuffle points ========
        noise = perturbation_factor * domain_scale * torch.randn_like(X_torch)
        perturbed_points = (X_torch + noise).clamp(X_torch.min(0).values, X_torch.max(0).values)
        shuffled_points = perturbed_points[torch.randperm(N)]
        shuffled_points.requires_grad = True

        total_epoch_loss = 0.0

        # ======== Minibatch ========
        for batch_idx in range(num_batches):
            start = batch_idx * minibatch_size
            end = start + minibatch_size
            x_batch = shuffled_points[start:end]

            optimizer.zero_grad()
            u, _ = model(x_batch)
            u = u.view(-1, 1)

            # Rayleigh quotient
            Lu = torch.sparse.mm(L_torch, u)
            Mu = torch.sparse.mm(M_torch, u)
            numerator = torch.sum(u * Lu)
            denominator = torch.sum(u * Mu) + 1e-8
            eigenvalue = numerator / denominator

            # Normalized residual
            eig_residual = Lu - eigenvalue * Mu
            eig_loss = torch.mean(eig_residual ** 2) / (torch.mean(u ** 2) + 1e-8)

            # Normalization
            norm_loss = (torch.sum(u * Mu) - 1.0) ** 2

            # Orthogonality
            ortho_loss = compute_orthogonality_loss(u, eigenfunctions, M_torch)

            total_loss = eig_loss + norm_loss + ortho_weight * ortho_loss
            total_loss.backward()
            optimizer.step()
            total_epoch_loss += total_loss.item()

        # ======== EMA slope convergence detection ========
        avg_loss = total_epoch_loss / num_batches
        if prev_loss is not None:
            ema_slope = 0.75 * ema_slope + 0.25 * abs(prev_loss - avg_loss)
        prev_loss = avg_loss

        # ======== Adaptive reinitialization ========
        trigger_reweight = (ema_slope < convergence_threshold and ema_slope > 0 and epoch > 2000)

        if trigger_reweight:
            # 1️⃣ Store converged eigenfunction first
            with torch.no_grad():
                u_full, _ = model(X_torch)
                u_full = u_full.view(-1, 1)
                Lu_full = torch.sparse.mm(L_torch, u_full)
                Mu_full = torch.sparse.mm(M_torch, u_full)
                λ_full = torch.sum(u_full * Lu_full) / (torch.sum(u_full * Mu_full) + 1e-8)

                eigenvalues.append(λ_full.item())
                eigenfunctions.append(u_full.detach())
                all_models.append(copy.deepcopy(model))

            # 2️⃣ Reinitialize weights for next eigenfunction
            model.apply(initialize_weights)
            eig_counter += 1
            print(f"Epoch {epoch} [Adaptive Reweight] | Eigenfunctions found: {eig_counter}")

            # 3️⃣ Stop training if enough eigenfunctions found
            if eig_counter >= num_eigenfunctions:
                print(f"All {num_eigenfunctions} eigenfunctions found. Stopping training.")
                break

        # ======== Logging ========
        if epoch % 500 == 0:
            print(f"Epoch {epoch:5d} | λ ≈ {eigenvalue.item():.6f} | Loss={avg_loss:.2e} | EMA slope={ema_slope:.2e}")
        if epoch % 100 == 0:
            loss_history['total'].append(avg_loss)
            loss_history['eig'].append(eig_loss.item())
            loss_history['norm'].append(norm_loss.item())
            loss_history['ortho'].append(ortho_loss.item())

    # ======== Return results ========
    return eigenvalues, eigenfunctions, all_models, loss_history

In [17]:
m = Mesh('bunny.obj')

centroid = m.verts.mean(0)
std_max = m.verts.std(0).max()

X = (m.verts - centroid)/std_max

m = Mesh(verts = X, connectivity = m.connectivity)

L, M = robust_laplacian.point_cloud_laplacian(X)

print('Computing Laplacian')
K_igl, M_igl = m.computeLaplacian()

# following Finite Elements methodology 
# K is stiffness matrix, M is mass matrix
# The problem to solve becomes 
# K*u = lambda * M*u
print('Computing eigen values')
eigvals, eigvecs = eigh(K_igl,M_igl)

# send all relevant numpy arrays to torch tensors
K_ = torch.from_numpy(K_igl).float().to(device)
M_ = torch.from_numpy(M_igl).float().to(device)
X_ = torch.from_numpy(m.verts).float().to(device)



eigenvalues, eigenfunctions, models, history = train_eigenvalue_pinn(
    X_, K_, M_, 
    hidden_dim=128, 
    epochs=5000, 
    lr=1e-2, 
    num_eigenfunctions=10
)

print("\n" + "="*60)
print("RESULTS")
print("="*60)
for i, lam in enumerate(eigenvalues):
    print(f"λ_{i} = {lam:.6f}")

Computing Laplacian
Computing eigen values
Training on device: cuda
Total points: 2503 | Minibatch size: 2503 | Batches per epoch: 1

Training eigenfunction 1/10
Epoch     0 | λ ≈ 328.459198 | Loss=3.16e+01 | EMA slope=1.00e+00
Epoch   500 | λ ≈ 345.488892 | Loss=3.38e+01 | EMA slope=4.33e+00
Epoch  1000 | λ ≈ 389.421600 | Loss=3.19e+01 | EMA slope=2.39e+00
Epoch  1500 | λ ≈ 379.023010 | Loss=2.97e+01 | EMA slope=1.48e+00
Epoch  2000 | λ ≈ 363.382111 | Loss=2.89e+01 | EMA slope=1.38e+00
Epoch  2500 | λ ≈ 377.447205 | Loss=3.11e+01 | EMA slope=2.78e+00
Epoch  3000 | λ ≈ 378.192871 | Loss=3.01e+01 | EMA slope=2.37e+00
Epoch  3500 | λ ≈ 394.451294 | Loss=2.85e+01 | EMA slope=1.88e+00
Epoch  4000 | λ ≈ 388.962158 | Loss=2.60e+01 | EMA slope=3.04e+00
Epoch  4500 | λ ≈ 408.374176 | Loss=2.87e+01 | EMA slope=1.60e+00
→ Found eigenvalue λ_0 = 267.581848

Training eigenfunction 2/10
Epoch     0 | λ ≈ 313.996216 | Loss=2.78e+01 | EMA slope=1.00e+00
Epoch   500 | λ ≈ 395.967621 | Loss=5.01e+01 | 

KeyboardInterrupt: 

In [10]:
print(f"The true Eigenvalues are: {np.array2string(eigvals[:10], formatter={'float': lambda x: f'{x:.3f}'})}")
print(f"The pred Eigenvalues are: {np.array2string(np.array(eigenvalues), formatter={'float': lambda x: f'{x:.3f}'})}")

The true Eigenvalues are: [0.000 0.160 0.425 0.438 0.538 0.612 0.896 1.274 1.496 1.643]
The pred Eigenvalues are: [0.000 0.015 0.043 0.151 0.383 0.386 0.492 0.387 0.341 0.332]


In [ ]:
print(f"The true Eigenvalues are: {np.array2string(eigvals[:10], formatter={'float': lambda x: f'{x:.3f}'})}")
print(f"The pred Eigenvalues are: {np.array2string(np.array(eigenvalues), formatter={'float': lambda x: f'{x:.3f}'})}")